In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
import pinecone 
import os
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import config

/Users/swimmingcircle/Code/streamlit_test/streamlit_venv/lib/python3.9/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
# initialize pinecone
pinecone_api_key = 'ad71b023-81cd-4946-bc21-2c18b51f9e6a'
pinecone.init(
    api_key=pinecone_api_key,  # find at app.pinecone.io
    environment='asia-southeast1-gcp-free'  # next to api key in console
)

In [5]:
openai_api_key = config.api_keys['openai_api_image_key']
# %%
embeddings = OpenAIEmbeddings(disallowed_special=(), openai_api_key=openai_api_key)

from langchain.document_loaders import TextLoader
loader = TextLoader('./sample.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, 
    chunk_overlap = 50,
    length_function = len)

docs = text_splitter.split_documents(documents) # list of lists of strings


In [7]:
index_name = 'tim-urban-test'
docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [8]:
query = "What is the meaning of life?"
docs = docsearch.similarity_search(query)

In [9]:
store = docs[0].page_content

In [10]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k-0613', openai_api_key=openai_api_key)

In [11]:
template ='''
Use the following pieces of context from waitbutwhy to answer the question at the end. 
If you don't know the answer, just clarify that you are not sure, but this might be how Tim Urban thinks.
'''
user_input = '''Write a post about the meaning of life within 300 words like waitbutwhy using "you" in a casual language'''
final_input = template+ store+ user_input

In [28]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(model_name='gpt-3.5-turbo-16k-0613', openai_api_key=openai_api_key)   


messages = [
    SystemMessage(content=template+store),
    HumanMessage(content=user_input)
]
response=chat(messages)

print(response.content,end='\n')

Hey there, you! Ever wondered about the meaning of life? Yeah, it's a big question, and trust me, you're not alone in pondering it. Even the brilliant Elon Musk has some thoughts on the matter.

You see, Elon is known for his mind-boggling advancements in technology and space exploration. But he also has a knack for simplifying complex concepts. Take his take on fear, for example. As a kid, he was scared of the dark, just like many of us. But when he realized that darkness is simply the absence of light, it seemed silly to be afraid of something that wasn't even there. And just like that, his fear vanished.

Now, let's bring this back to the meaning of life. It's easy to get caught up in the grandeur of it all, wondering about our purpose and what it's all about. But here's a thought: what if the meaning of life is simply about embracing the moments that truly matter to you?

Think about it. Elon, with all his incredible achievements, realizes the importance of slowing down as his kids